# OpenAI APIの前準備

In [ ]:
# パッケージのインストール
!pip install -U openai

In [ ]:
import os
from google.colab import userdata

# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

# 画像生成

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input="かわいい猫耳メイドの絵を作成して",
    tools=[{"type": "image_generation"}],
)

In [ ]:
import base64

# 画像ファイルの保存
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]
if image_data:
    image_base64 = image_data[0]
    with open("output.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

# マルチターン画像生成

In [ ]:
# 2ターン目の推論の実行
response_fwup = client.responses.create(
    model="gpt-5",
    previous_response_id=response.id, # レスポンスID
    input="カフェで働いているところを描いて。横長画像で。",
    tools=[{"type": "image_generation"}],
)

In [ ]:
# 画像ファイルの保存
image_data_fwup = [
    output.result
    for output in response_fwup.output
    if output.type == "image_generation_call"
]
if image_data_fwup:
    image_base64 = image_data_fwup[0]
    with open("output2.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

# ストリーミング

In [ ]:
# ストリーミングで推論を実行
stream = client.responses.create(
    model="gpt-5",
    input="ネコが日向ぼっこしてるイラストを描いて",
    stream=True,
    tools=[{"type": "image_generation", "partial_images": 2}],
)

In [ ]:
import base64

# 画像ファイルの保存
for event in stream:
    if event.type == "response.image_generation_call.partial_image":
        idx = event.partial_image_index
        image_base64 = event.partial_image_b64
        image_bytes = base64.b64decode(image_base64)
        with open(f"streaming{idx}.png", "wb") as f:
            f.write(image_bytes)

# 修正プロンプト

In [ ]:
# 修正プロンプトの確認
for item in response.output:
    # image_generation_call
    if item.type == "image_generation_call":
        print("==image_generation_call==")
        print(item.revised_prompt)